<a href="https://colab.research.google.com/github/arun8166/BCS_tasks/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
!pip install keras
!pip install tensorflow
!pip install rasa==1.1.4

In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import scipy
from PIL import Image
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops

In [ ]:
import opendatasets as od
import pandas as pd

od.download("https://www.kaggle.com/datasets/koryakinp/fingers")

In [6]:
import os
import cv2
import numpy as np

# Path to the dataset directory
dataset_dir = '/content/fingers/test'

# Initialize lists to store images and labels
images = []
labels = []

# Iterate through each image file
for filename in os.listdir(dataset_dir):
    # Load the image using OpenCV
    image = cv2.imread(os.path.join(dataset_dir, filename), cv2.IMREAD_GRAYSCALE)
    
    # Resize the image to a desired size (e.g., 128x128)
    image = cv2.resize(image, (64, 64,))
    
    # Append the image to the images list
    images.append(image)
    
    # Get the label from the filename (the first character)
    label = int(filename[-6])
    
    # Append the label to the labels list
    labels.append(label)

# Convert the lists to NumPy arrays
x_test = np.array(images)
y_test = np.array(labels)


In [7]:
# Path to the dataset directory
dataset_dir = '/content/fingers/train'

# Initialize lists to store images and labels
images = []
labels = []

# Iterate through each image file
for filename in os.listdir(dataset_dir):
    # Load the image using OpenCV
    image = cv2.imread(os.path.join(dataset_dir, filename), cv2.IMREAD_GRAYSCALE)
    
    # Resize the image to a desired size (e.g., 128x128)
    image = cv2.resize(image, (64, 64,))
    
    # Append the image to the images list
    images.append(image)
    
    # Get the label from the filename (the first character)
    label = int(filename[-6])
    
    # Append the label to the labels list
    labels.append(label)

# Convert the lists to NumPy arrays
x_train = np.array(images)
y_train = np.array(labels)

In [8]:
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

In [9]:
from tensorflow.keras.utils import to_categorical


Y_train = to_categorical(y_train, num_classes=6)

Y_test = to_categorical(y_test, num_classes=6)

In [ ]:
print ("Y_train shape: " + str(Y_train.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [11]:
def convolutional_model(input_shape):

    input_img = tf.keras.Input(shape=input_shape)
    ## CONV2D: 8 filters 4x4, stride of 1, padding 'SAME'
    Z1 = tfl.Conv2D(filters= 8. , kernel_size=4 , padding='same',strides=1)(input_img)
    ## RELU
    A1 = tfl.ReLU()(Z1)
    ## MAXPOOL: window 8x8, stride 8, padding 'SAME'
    P1 = tfl.MaxPool2D(pool_size=8, strides=8, padding='SAME')(A1)
    ## CONV2D: 16 filters 2x2, stride 1, padding 'SAME'
    Z2 = tfl.Conv2D(filters= 16. , kernel_size=2 , padding='same',strides=1)(P1)
    ## RELU
    A2 =  tfl.ReLU()(Z2)
    ## MAXPOOL: window 4x4, stride 4, padding 'SAME'
    P2 = tfl.MaxPool2D(pool_size=4, strides=4, padding='SAME')(A2)
    ## FLATTEN
    F = tfl.Flatten()(P2)
    ## Dense layer
    ## 6 neurons in output layer. Hint: one of the arguments should be "activation='softmax'" 
    outputs = tfl.Dense(units= 6 , activation='softmax')(F)
    model = tf.keras.Model(inputs=input_img, outputs=outputs)
    return model

In [ ]:
conv_model = convolutional_model((64, 64,1))
conv_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
conv_model.summary()

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train[0:1800,], Y_train[0:1800,]))
train_dataset = train_dataset.batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test[0:300,], Y_test[0:300,]))
test_dataset = test_dataset.batch(64)
history = conv_model.fit(train_dataset, epochs=100, validation_data=test_dataset)

In [ ]:
import random
import cv2
from matplotlib import pyplot as plt

numbers = list(range(1, 1001))
random_array = random.sample(numbers, 10)

fig = plt.figure(figsize=(10, 7))

for i, idx in enumerate(random_array):
    img_resized = cv2.resize(x_test[idx], (64, 64))
    y_pred = np.argmax(conv_model.predict(np.expand_dims(x_test[idx], axis=0),verbose=0))
    ax = fig.add_subplot(1, 10, i+1)
    ax.imshow(img_resized, cmap='gray')
    ax.axis('off')
    ax.set_title("pred = " + str(y_pred))

plt.tight_layout()
plt.show()
